In [1]:
from src.mazeData import *

In [2]:
n=7
maze1=makeMaze(n)
mazeAvailActs=defineMazeAvailableActions(maze1)

In [3]:
maze1

array([[1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 1, 0, 1],
       [1, 1, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 1, 0, 1],
       [1, 1, 1, 1, 1, 1, 0]])

In [4]:
mazeAvailActs

{(0, 0): ['right'],
 (0, 1): ['left', 'right', 'down'],
 (0, 2): ['left', 'right', 'down'],
 (0, 3): ['left', 'right', 'down'],
 (0, 4): ['left', 'right', 'down'],
 (0, 5): ['left', 'right', 'down'],
 (0, 6): ['left', 'down'],
 (1, 0): [],
 (1, 1): ['up', 'right', 'down'],
 (1, 2): ['left', 'up', 'right', 'down'],
 (1, 3): ['left', 'up', 'right'],
 (1, 4): ['left', 'up', 'right', 'down'],
 (1, 5): ['left', 'up', 'right', 'down'],
 (1, 6): ['left', 'up', 'down'],
 (2, 0): ['right'],
 (2, 1): ['left', 'up', 'right', 'down'],
 (2, 2): ['left', 'up', 'down'],
 (2, 3): [],
 (2, 4): ['up', 'right', 'down'],
 (2, 5): ['left', 'up', 'right'],
 (2, 6): ['left', 'up', 'down'],
 (3, 0): [],
 (3, 1): ['up', 'right', 'down'],
 (3, 2): ['left', 'up', 'right'],
 (3, 3): ['left', 'right', 'down'],
 (3, 4): ['left', 'up', 'down'],
 (3, 5): [],
 (3, 6): ['up'],
 (4, 0): ['right'],
 (4, 1): ['left', 'up', 'down'],
 (4, 2): [],
 (4, 3): ['up', 'right'],
 (4, 4): ['left', 'up', 'down'],
 (4, 5): [],
 (4, 6